### 作業
目前已經學過許多的模型，相信大家對整體流程應該比較掌握了，這次作業請改用**手寫辨識資料集**，步驟流程都是一樣的，請試著自己撰寫程式碼來完成所有步驟

In [3]:
from sklearn import datasets, metrics
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [4]:
digits = datasets.load_digits()

In [5]:
x_train, x_test, y_train, y_test = train_test_split(digits.data ,digits.target, test_size=0.25)

In [7]:
clf = GradientBoostingClassifier()

In [9]:
clf.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [11]:
y_pred = clf.predict(x_test)

In [12]:
metrics.accuracy_score(y_test, y_pred)

0.96

### GBDT調參
https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/

#### 三種參數

##### 1.Tree-Specific Parameters
- min_samples_split : 樹split時要求的最少樣本數，可以用來避免over fitting，當這個值太高時，有可能會under fitting，tune by cv。
- min_samples_leaf : 類似min_samples_split，樹split後，每個葉節點要求的最少樣本數，一樣避免over fitting。需要注意的是，當有imbalance label的問題時，這個值就不能設太大，因為少數label分裂後於葉末節點的數目不會太多。
- max_depth : 樹的深度(控制over fitting)，很深的深度可以讓型學習到特定樣本的特徵。
- max_leaf_nodes : 最大業末節點數目，就是2^(max depth)，若用此參數，就會忽略max depth。
- max_features : 分裂時會用的最大feature數，一般來說，總樣本數開根號可以得到不錯效果，通常需要檢測到30~40%。但這是以經驗來說，模型設定終究要取決於你的資料集。

##### 2.Boosting Parameters
- learning rate : 越小理論上模型會越robust，但通常需要建構更多的樹 。
- n_estimators : 在給定learning rate後，用cv來找。
- subsample : 建構每一棵樹時，用的樣本數，通常0.8左右就有不做的效果，但也可以用cv tune更好的比例。

##### 3.Miscellaneous Parameters：一些其他雜項調整運作的參數。
- loss : 每一個split時，要minimiaze的function。
- init : 初始化模型的output。當我們有train其他模型，其他模型的output可以當作GBDT output的初始值。
- random_state : 設定random_state才能固定隨機性，以比較不同參數的差異。
- verbose : 模型擬合時，顯示的資訊量。
- warm_start : 可以節省很多trian的時間!?詳細用法可能要看原始文檔。

### 調參順序
##### 1.決定初始learning rate & n_estimate

-找一個較高的初始學習速率，通常0.1，0.05~0.2 case by case。

-在這個學習速率下找到最佳的n_estimate，40-70，通常會找一個較小的數目，以利之後CV找最佳參數的執行速度。


##### 2. 設定以下參數的起始值，然後先找1.說的n_estimate。(若lr=0.1 -> n=20，可以嘗試調低的learning rate。若lr = 0.1 -> n=100，可以嘗試調高learning rate，通常將她控制在 40-70之間，讓後面調整參數過程不會太久)

-min_samples_split : 通常總樣本數的0.5-1%。有imbalalnce sample問題時，可以設更小。

-min_samples_leaf : 可以依直覺設，依樣imbalance，時可以用較小數字。

-max_depth : 通常5-8。

-max_features : 'sqrt'

-subsample = 0.8

##### 3. 決定了learning rate及n_estimate後，開始調其他參數。
-Tune max_depth and min_samples_split 

-Tune min_samples_leaf 

-Tune max_features 

ex:
-> {'max_depth':range(5,16,2), 'min_samples_split':range(200,1001,200)}

-> {'min_samples_split':range(1000,2100,200), 'min_samples_leaf':range(30,71,10)} #min_samples_split最後結果碰到上界，繼續往下測。

-> {'max_features':range(7,20,2)}

##### 4. tune subsample並降低learning rate

ex: 

-> {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9]}
###### 最後 - 降低learnin rate -> 例如從0.1->0.05，降低一半，此時，n_estimator應該要上升2倍 60->120，用這樣的概念去做最後的調整，找出最佳的learning rate以及n_estimate。
